In [5]:
import numpy as np
from random import randint, gauss, Random
import json
import os
from PIL import Image
from joblib import Parallel, delayed
import multiprocessing as mp

xsize = 600
ysize = 600
mu = 4
sigma = 3
num = 100
maxHits = 15
folder = "prob"
listName = "list"


def getHit(local_random):

    x = local_random.randint(0, xsize)
    y = local_random.randint(0, ysize)
    r = round(abs(local_random.gauss(mu=mu, sigma=sigma)), 2)

    return (x, y, r)


def drawImage(num):

    fileName = "{:06d}".format(num)

    # Create a 600x600x3 array of 8 bit unsigned integers; make them white
    data = np.full((xsize, ysize, 3), (255, 255, 255), dtype=np.uint8)

    x = []
    y = []
    r = []

    local_random = Random()
    local_random.seed(num)

    hits = local_random.randint(1, maxHits)

    for h in range(hits):
        hit = getHit(local_random)
        x.append(hit[0])
        y.append(hit[1])
        r.append(hit[2])

        for i in range(xsize):
            for j in range(ysize):
                if (i-x[h])**2+(j-y[h])**2 <= r[h]**2:
                    data[i, j] = [0, 0, 0]

    img = Image.fromarray(data)
    img.save(folder+"/"+fileName+".png")

    imgData = {
        "num": fileName,
        "hits": hits,
        "x": x,
        "y": y,
        "r": r
    }
    with open(folder+"/"+listName+".json", mode='ab+') as handle:
        handle.seek(-1, os.SEEK_END)
        handle.truncate()

    with open(folder+"/"+listName+".json", mode='a+', encoding='utf-8') as handle:
        if fileName != "000000":
            handle.write(',')
        json.dump(imgData, handle, indent=2)
        handle.write(']')


def main():

    with open(folder+"/"+listName+".json", mode='w', encoding='utf-8') as handle:
        empty = []
        json.dump(empty, handle)
        print("file "+folder+"/"+listName+".json created")

    num_cores = 1
    # Comment the following if you do not want run in CPU parallel mode.
   # num_cores = mp.cpu_count()

    drawImage(0)

    Parallel(n_jobs=num_cores)(delayed(drawImage)(i) for i in range(1, num))


main()

file prob/list.json created
